### Load ground truth

In [1]:
folder = "../../KOI_Data/"
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_sets = pd.read_csv(folder+"/koi_sets_unb.csv") 
mask_train = (df_sets["Set"] == "Train").values
mask_test = (df_sets["Set"] == "Test").values
mask_unlabeled = (df_sets["Set"] == "Unlabeled").values


df_meta = pd.read_csv(folder+"/koi_metadata.csv")
df_meta_train = df_meta[mask_train]
df_meta_test = df_meta[mask_test]
df_meta_unb = df_meta[mask_test]

y_train = ((df_meta_train["NExScI Disposition"]=="CONFIRMED")*1).values
y_test = ((df_meta_train["NExScI Disposition"]=="CONFIRMED")*1).values

N_train = y_train.shape[0]
N_test = y_test.shape[0]
y_train

array([0, 0, 0, ..., 1, 1, 0])

### Simmulate data (light curves)

In [2]:
import numpy as np

T = 70000

X_train = np.random.rand(N_train,T)
X_test = np.random.rand(N_test,T)
X_train.shape

(4692, 70000)

In [3]:
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input, Conv1D, Dense, Flatten, MaxPool1D, Reshape, UpSampling1D, Lambda
from keras.layers import GlobalAveragePooling1D,GlobalMaxPool1D, TimeDistributed, GRU,LSTM, RepeatVector

#def Conv1DTranspose(input_tensor, filters, kernel_size, strides=2, padding='same',activation='linear'):
#    x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
#    x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1), padding=padding,activation=activation)(x)
#    x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
#    return x

Using TensorFlow backend.


In [4]:
#need data with extra dim
X_train = np.expand_dims(X_train,axis=-1)
X_test = np.expand_dims(X_test,axis=-1)

### Fully convolutional
---
Opciones a cambiar y realizar:
* AverageMaxPooling envez de GlobalMaxPooling
* Dropout
* BatchNormalization
* Conv con strides y Convtranspuesta envez de **max** pooling y upsampling
* más capas..

In [51]:
def encoder_model_CNN1D(input_dim, latent_dim, L=1, filters=8,kernel_s =10, pool=5):
    #it = Input(shape=(None,1))  #variable length..
    it = Input(shape=(input_dim,1))  #fixed length..
    f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(it)
    f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(f1)
    f1 = MaxPool1D(pool)(f1) #minpooling?? -- stride..
    #f1 = Conv1D(filters, kernel_s, strides=pool, padding='same')(f1)
    for _ in range(L-1):
        filters = int(filters*2)
        f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(f1)
        f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(f1)
        f1 = MaxPool1D(pool)(f1)
        #f1 = Conv1D(filters, kernel_s, strides=pool, padding='same')(f1)
    
    redim_shape = K.int_shape(f1)[1:] 
    d1 = GlobalMaxPool1D()(f1) #or global maxpooling
    d1 = Dense(512,activation='relu')(d1)
    out_latent= Dense(latent_dim, activation='linear')(d1)
    return Model(inputs=it, outputs=out_latent), redim_shape

#Conv2d!!!
from keras.layers import Conv2D, Conv2DTranspose, MaxPool2D,UpSampling2D
def decoder_model_CNN2D(input_dim, redim_shape, L=1, filters=8, kernel_s=10, pool=5):
    it = Input(shape=(input_dim,))
    d1 = Dense(512, activation='relu')(it)
    d1 = Dense(redim_shape[-1], activation='relu')(d1)
    f1 = RepeatVector(redim_shape[0])(d1) #inverso a maxpooling :/
    
    #agregar dim extra para procesar por conv2d
    f1 = Lambda(lambda x: K.expand_dims(x, axis=2))(f1) #along channel axis
    
    filters = int(filters*2**(L-1))
    for _ in range(L):
        f1 = UpSampling2D((pool,1))(f1)
        f1 = Conv2D(filters, (kernel_s,1), strides=1, padding='same', activation='relu')(f1)
        f1 = Conv2D(filters, (kernel_s,1), strides=1, padding='same', activation='relu')(f1)
        filters = int(filters/2)
    out_x = Conv2D(1, (kernel_s,1), strides=1, padding='same', activation='linear')(f1)
    out_x = Lambda(lambda x: K.squeeze(x, axis=-1))(out_x)
    return Model(inputs=it, outputs=out_x)


""" TO SLOW: DEPRECATED"""
def decoder_model_CNN1D(input_dim, redim_shape, L=1, filters=8, kernel_s=10, pool=5):
    it = Input(shape=(input_dim,))
    d1 = Dense(512, activation='relu')(it)
    d1 = Dense(int(np.prod(redim_shape)), activation='relu')(d1)
    f1 = Reshape(redim_shape)(d1)
    
    filters = int(filters*2**(L-1))
    for _ in range(L):
        f1 = UpSampling1D(pool)(f1)
        f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(f1)
        f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(f1)
        filters = int(filters/2)
    out_x = Conv1D(1, kernel_s, strides=1, padding='same', activation='linear')(f1)
    return Model(inputs=it, outputs=out_x)


### convolutional and then rnn (as text)

In [63]:
def encoder_model_CNNRNN(input_dim, latent_dim, L1=1, filters=8,kernel_s =10, pool=5, L2=1, units=32):
    it = Input(shape=(input_dim,1)) 
    f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(it)
    f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(f1)
    f1 = MaxPool1D(pool)(f1)
    for _ in range(L1-1):
        filters = int(filters*2)
        f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(f1)
        f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(f1)
        f1 = MaxPool1D(pool)(f1)

    for _ in range(L2):
        f1 = GRU(units,return_sequences=True)(f1)
        units = int(units*2)
    redim_shape = K.int_shape(f1)[1:] 

    #d1 = GlobalMaxPool1D()(f1) #or global maxpooling
    d1 = GRU(int(units/2),return_sequences=False)(f1)
    
    d1 = Dense(512,activation='relu')(d1)
    out_latent= Dense(latent_dim, activation='linear')(d1)
    return Model(inputs=it, outputs=out_latent), redim_shape

#... falta decoder..

from keras.layers import Conv2D, Conv2DTranspose, MaxPool2D,UpSampling2D
def decoder_model_CNNRNN(input_dim, redim_shape, L1=1, filters=8,kernel_s =10, pool=5, L2=1, units=32):
    it = Input(shape=(input_dim,))
    d1 = Dense(512, activation='relu')(it)
    d1 = Dense(redim_shape[-1], activation='relu')(d1)
    f1 = RepeatVector(redim_shape[0])(d1) #inverso a maxpooling :/
    
    units = int(units*2**(L2-1))
    for _ in range(L2):
        f1 = GRU(units,return_sequences=True)(f1)
        units = int(units/2)
            
    #agregar dim extra para procesar por conv2d
    f1 = Lambda(lambda x: K.expand_dims(x, axis=2))(f1) #along channel axis
    
    filters = int(filters*2**(L2-1))
    for _ in range(L2):
        f1 = UpSampling2D((pool,1))(f1)
        f1 = Conv2D(filters, (kernel_s,1), strides=1, padding='same', activation='relu')(f1)
        f1 = Conv2D(filters, (kernel_s,1), strides=1, padding='same', activation='relu')(f1)
        filters = int(filters/2)
    out_x = Conv2D(1, (kernel_s,1), strides=1, padding='same', activation='linear')(f1)
    out_x = Lambda(lambda x: K.squeeze(x, axis=-1))(out_x)
    return Model(inputs=it, outputs=out_x)


### RNN jerárquica

In [25]:
def encoder_model(input_dim, latent_dim, L=1, units=8):
    it = Input(shape=(input_dim,1))
    #for level in range(Level:
    
    splits = 10
    T_w = int(input_dim/splits) #y si sobra?...
    f1 = Reshape([splits, T_w, 1])(it)
    
    aux_units = units
    for _ in range(L):
        f1 = TimeDistributed(GRU(aux_units, return_sequences=True))(f1)
        aux_units = int(aux_units*2)
    #terminar..
    f1 = TimeDistributed(GRU(int(units/2), return_sequences=False))(f1)
    
    #ultimo nivel..
    aux_units = units
    for _ in range(L):
        f1 = GRU(aux_units, return_sequences=True)(f1)
        aux_units = int(aux_units*2)

    redim_shape = (splits, int(f1.shape[2]))
    redim_shape = K.int_shape(f1)[1:] 
    #terminar..
    d1 = GRU(int(aux_units/2), return_sequences=False)(f1)
    d1 = Dense(512, activation='relu')(d1)
    out_latent= Dense(latent_dim, activation='linear')(d1)
    return Model(inputs=it, outputs=out_latent), redim_shape

### Dynamic K-max pooling

In [34]:
from keras.engine import Layer, InputSpec
import tensorflow as tf

class KMaxPooling(Layer):
    """
    K-max pooling layer that extracts the k-highest activations from a sequence (2nd dimension). TensorFlow backend.
    """
    def __init__(self, k=1, **kwargs):
        super().__init__(**kwargs)
        self.input_spec = InputSpec(ndim=3)
        self.k = k

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.k, input_shape[2])

    def call(self, inputs):
        # swap last two dimensions since top_k will be applied along the last dimension
        shifted_input = tf.transpose(inputs, [0, 2, 1])
        # extract top_k, returns two tensors [values, indices]
        top_k = tf.nn.top_k(shifted_input, k=self.k, sorted=True, name=None)[0] 
        return tf.transpose(top_k, [0, 2, 1])
    
class KMinPooling(Layer):
    """
    K-max pooling layer that extracts the k-highest activations from a sequence (2nd dimension). TensorFlow backend.
    """
    def __init__(self, k=1, **kwargs):
        super().__init__(**kwargs)
        self.input_spec = InputSpec(ndim=3)
        self.k = k

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.k, input_shape[2])

    def call(self, inputs):
        # swap last two dimensions since top_k will be applied along the last dimension
        shifted_input = tf.transpose(inputs, [0, 2, 1])        
        # extract top_k, returns two tensors [values, indices]
        top_k = -tf.nn.top_k(-shifted_input, k=self.k, sorted=True, name=None)[0] #min
        return tf.transpose(top_k, [0, 2, 1])
    
import math
def K_l(L,seq_len,k_top,layer=1): #dinamyc k-max
    return max(k_top, math.ceil( seq_len*(L-layer)/L) )

def encoder_model_KPool(input_dim, latent_dim, L=1, filters=8,kernel_s =10, K_top=500):
    #k_top = 500 #podría ser el periodo más corto entre los datos...-- largo final de codificacion conv...
    it = Input(shape=(input_dim,1))  #None...
    f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(it)
    f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(f1)
    f1 = KMinPooling(k = K_l(L, input_dim, K_top, layer=1))(f1)
    for l in range(L-1):
        filters = int(filters*2)
        f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(f1)
        f1 = Conv1D(filters, kernel_s, strides=1, padding='same', activation='relu')(f1)
        f1 = KMinPooling(k = K_l(L, input_dim, K_top, layer=l+2))(f1) #como se desencodea??
    redim_shape = K.int_shape(f1)[1:] 
    
    #DUDA DE CÓMO HACER FLATTEN O GLOBAL POOL..
    #f1 = GlobalMaxPool1D()(f1) #or global maxpooling--inverse es repeat vector..
    f1 = Flatten()(f1)  #--- OJO CON EL DECODER..
    
    d1 = Dense(512,activation='relu')(f1)
    out_latent= Dense(latent_dim, activation='linear')(d1)
    return Model(inputs=it, outputs=out_latent), redim_shape

    
#SE NECESITA UN DECODER...

### build Encoder

In [73]:
#encoder, redim_shape = encoder_model_CNN2D(T, 32, L=3, filters=8, pool=5, kernel_s=10) -- deprecated
#encoder, redim_shape = encoder_model_CNN1D(T, 32, L=3, filters=8, pool=5, kernel_s=10)
#encoder, redim_shape = encoder_model_CNNRNN(T, 32, L1=3, filters=8, pool=5, kernel_s=10, L2=3, units=32)
encoder, redim_shape = encoder_model_KPool(T, 32, L=3, filters=8, K_top=100)
#encoder, redim_shape = encoder_model(T, 32, L=3, units=8)

encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        (None, 70000, 1)          0         
_________________________________________________________________
conv1d_73 (Conv1D)           (None, 70000, 8)          88        
_________________________________________________________________
conv1d_74 (Conv1D)           (None, 70000, 8)          648       
_________________________________________________________________
k_min_pooling_22 (KMinPoolin (None, 46667, 8)          0         
_________________________________________________________________
conv1d_75 (Conv1D)           (None, 46667, 16)         1296      
_________________________________________________________________
conv1d_76 (Conv1D)           (None, 46667, 16)         2576      
_________________________________________________________________
k_min_pooling_23 (KMinPoolin (None, 23334, 16)         0         
__________

In [74]:
70000/46667

1.4999892857908157

In [75]:
46667/23334

1.9999571440815977

In [76]:
23334/100

233.34

### build Decoder

In [64]:
#decoder = decoder_model_CNN2D(32, redim_shape, L=3, filters=8, pool=5, kernel_s=10)
decoder = decoder_model_CNNRNN(32, redim_shape, L1=3, filters=8,kernel_s =10, pool=5, L2=3, units=32)

#decoder = decoder_model_CNN1D(32, redim_shape, L=3, filters=8, pool=5) --deprecated

decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        (None, 32)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 512)               16896     
_________________________________________________________________
dense_50 (Dense)             (None, 128)               65664     
_________________________________________________________________
repeat_vector_13 (RepeatVect (None, 560, 128)          0         
_________________________________________________________________
gru_16 (GRU)                 (None, 560, 128)          98688     
_________________________________________________________________
gru_17 (GRU)                 (None, 560, 64)           37056     
_________________________________________________________________
gru_18 (GRU)                 (None, 560, 32)           9312      
__________

### build Autoencoder

In [65]:
it = Input(shape=X_train.shape[1:])
out = decoder(encoder(it))
autoencoder = Model(it,out)
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        (None, 70000, 1)          0         
_________________________________________________________________
model_28 (Model)             (None, 32)                300160    
_________________________________________________________________
model_30 (Model)             (None, 70000, 1)          257889    
Total params: 558,049
Trainable params: 558,049
Non-trainable params: 0
_________________________________________________________________


In [66]:
#quizas sea necesario definir una loss ...
autoencoder.compile(loss='mse',optimizer='adam') #focal loss?

In [67]:
def train_model(model,X,y,epochs=1,batch_size=32):
    return model.fit(X,y, epochs=epochs, batch_size=batch_size, validation_split=0.2)

In [14]:
train_model(autoencoder, X_train, X_train,batch_size=256) #conv 2d--conv2d

Train on 3753 samples, validate on 939 samples
Epoch 1/1
3753/3753 [==============================] - 388s 103ms/step - loss: 0.2901 - val_loss: 0.2584


In [28]:
train_model(autoencoder, X_train_a, X_train, batch_size=256) #conv1d--conv2d

Train on 3753 samples, validate on 939 samples
Epoch 1/1
3753/3753 [==============================] - 376s 100ms/step - loss: 0.3179 - val_loss: 0.2975


In [13]:
train_model(autoencoder, X_train, X_train, batch_size=256) #conv1d--conv2d -- con global pool

Train on 3753 samples, validate on 939 samples
Epoch 1/1
3753/3753 [==============================] - 371s 99ms/step - loss: 0.1696 - val_loss: 0.0885


In [68]:
train_model(autoencoder, X_train, X_train, batch_size=256) #conv1dRNN -- RNNconv2d

Train on 3753 samples, validate on 939 samples
Epoch 1/1
3753/3753 [==============================] - 485s 129ms/step - loss: 0.2148 - val_loss: 0.1565


In [14]:
X_train_hat = autoencoder.predict(X_train)
X_train_encoder = encoder.predict(X_train)
X_train_hat.shape

(4692, 70000, 1)

In [15]:
X_train_encoder.shape

(4692, 32)

In [4]:
from evaluation import calculate_metrics
aux = calculate_metrics(y_train,np.ones((N_train,1)))

                F1 macro  F1 micro    F1 raw  F1 weighted  Precision raw  \
False Positive  0.274583  0.274583  0.000000     0.207868       0.000000   
Confirmed       0.274583  0.274583  0.549165     0.207868       0.378517   

                Recall raw  
False Positive         0.0  
Confirmed              1.0  


/users/fmena/anaconda3/envs/tesis/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/users/fmena/anaconda3/envs/tesis/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
